In [ ]:
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

df = pd.read_csv('data/Cleantech Media Dataset/cleantech_media_dataset_v2_2024-02-23.csv')
df.head()

# Preprocessing & Indexing

In [ ]:
import ast

df['content'] = df['content'].apply(ast.literal_eval)
df['content'] = df['content'].apply(lambda x: [str(i) for i in x])
df['content'] = df['content'].apply(lambda x: '\n\n'.join(x))

df['content'].head()

In [ ]:
from src.embedding_strategy import EmbeddingStrategy
strategy = EmbeddingStrategy.get_default_strategy()

documents = []
for index, row in tqdm(df.iterrows()):
    content = row['content']
    
    row = row.fillna('')
    
    metadata = {
        "url": row['url'],
        "domain": row['domain'],
        "title": row['title'],
        "author": row['author'],
        "date": row['date']
    }
    
    cleaned_content = strategy.processor.clean(content)
    row_docs = strategy.processor.chunk(content, metadata)
    
    documents += row_docs
    
documents = documents[:100] # Only use the first 100 documents for testing

In [ ]:
strategy.vector_store.add_documents(documents, verbose=True, batch_size=128)
strategy.vector_store.similarity_search("The company is also aiming to reduce gas flaring?")

# Retrieval & Generation

In [ ]:
from src.generator import Generator
import os

gen = Generator(openai_api_key=os.getenv("OPENAI_API_KEY"), 
                embedding_strategy=strategy)

gen.ask("What's fifty grand to a motherfucker like me?")

In [ ]:
retriever = strategy.retriever
retriever.invoke("Who was in Paris?")

In [ ]:
# Based on rag evaluation set 
result = retriever.get_relevant_documents("What's fifty grand to a motherfucker like me?")
print(result[0].page_content)
print(result[0].metadata)